# Part1 : Build a CNN

In [1]:
# import the keras libraries and packages
from keras.models import Sequential 
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


- Sequential : Initiate the NN, two ways: as a sequence of layers or as a graph. CNN is a sequence of layers, so we use Sequential to initialize our NN.
- Convolution2D : Images are 2D, videos are 3D with time. Here we deal with images. So it is Convolution2D.
- MaxPooling2D 
- Flatten : convert matric to a large vecter
- Dense : Fully connect layer

In [2]:
# Initialising the CNN
model = Sequential()

In [3]:
# Step1 : Convolution layer
model.add(Convolution2D(32,(3,3), input_shape=(64,64,3), activation='relu' ))
# 64 - numbers of filters (feature detectors, the numbers of feature maps)
# (3,3) - filters' dimension, or kernel size ,(5,5),(7,7)
# input_shape - Image size and numbers of channel
# activation - Relu nonlinear function, no negative
# others parameters just keep default: strides=(1, 1), padding='valid' 
# output : (62,62,64)

In [4]:
# Step2 : Pooling layer : in oder to reduce the numbers of input nodes before flatten this to a large vector
model.add(MaxPooling2D(pool_size=(2,2)))
# pool_size=(2, 2), maximium value of four (2X2 square) 
# strides=None, 
# padding='valid' 
# the model reduced by 2
# the output size = half of original size + 1 : (7,7)->(4,4) (5,5)->(3,3)   
# output : (31,31,64)

In [5]:
# Step3: Flatten
model.add(Flatten())
# convert all of the feature maps into a large feature vector, one dimensional vector. 
# output : 31*31*64 = 61504

In [6]:
# Step4 : Fully connection layer as hidden layer
model.add(Dense(activation="relu", units=128))
# units = 128 : the last step generated 61504 input nodes, the final output nodes just need 1, cat or dog. 
# How much nodes in hidden layer there is no rules. 
# As general, it is between input nodes(61504) and output nodes( 1, cat or dog ). 
# But to common practice, it is better to choose a power of 2, and around 100. 128 hidden nodes is better for this case.
# activation='relu'

# Output layer
model.add(Dense(units = 1, activation='sigmoid'))
# units = 1 : cat or dog
# activation='sigmoid' : it is because only have a binary outcome , if more than two categories, we need softmax 


In [7]:
# Compile the CNN
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# optimizer='adam', (sgd)
# loss='binary_crossentropy', if more than three outcomes, use category_crossentropy
# metrics=['accuracy']

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 30752)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               3936384   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 3,937,409
Trainable params: 3,937,409
Non-trainable params: 0
_________________________________________________________________


# Part2 : Fit the CNN to the images

In [9]:
# Step1: preprocessing the images
# Before fit the  model, proceed to image augmentation (consists of pre-processing images) to prevent overfitting
# data augmentation: random transformation on a random selection of images, like rotation, flipping, shifting, shearing...
# data augmentation functions: 1-enrich datasets without adding more images,
#                              2-reduce overfitting with a small amount of images

from keras.preprocessing.image import ImageDataGenerator

In [10]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

# rescale = 1./255: pixel value between 0-1
# shear_range = 0.2: Through a shear mapping, a rectangle becomes a parallelogram.
# zoom_range = 0.2:  Range for random zoom. 
# horizontal_flip = True

In [11]:
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

# target_size = (64, 64), the images size are different, so here they are made into 64 * 64. Every images are same size.
# (64,64) is set according to the input_shape in the first layer( step 1, part 1, convolutional layer).
# batch_size = 32, the size of the batches in which some random samples of images will be included
# batch_size contains the number of iamges will go through the CNN after weights updated. Every 32 images update weights once.
# class_mode = 'binary', two classes here, so binary.  "categorical", "binary", "sparse" or None. Default: "categorical".

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [12]:
# Step2: fit and test the performance
model.fit_generator(training_set,
                    steps_per_epoch = 8000,
                    epochs = 5,
                    validation_data = test_set,
                    validation_steps = 2000)
# generator, 
# steps_per_epoch, total numbers of images in training set 
# epochs=25, total number of iterations on the data.
# verbose=1, 0, 1, or 2.
# validation_data=test_set, a generator for the validation data, here is test_set
# validation_steps=None,  Total number of steps (batches of samples) to yield from  generator before stopping.

Epoch 1/5
8000/8000 [==============================] - 3201s - loss: 0.4087 - acc: 0.8064 - val_loss: 0.5636 - val_acc: 0.7887
Epoch 2/5
8000/8000 [==============================] - 3011s - loss: 0.1874 - acc: 0.9247 - val_loss: 0.8110 - val_acc: 0.7713
Epoch 3/5
8000/8000 [==============================] - 2954s - loss: 0.1109 - acc: 0.9586 - val_loss: 0.9859 - val_acc: 0.7727
Epoch 4/5
8000/8000 [==============================] - 3020s - loss: 0.0780 - acc: 0.9719 - val_loss: 1.0221 - val_acc: 0.7841
Epoch 5/5
8000/8000 [==============================] - 2875s - loss: 0.0598 - acc: 0.9790 - val_loss: 1.2114 - val_acc: 0.7801


In [13]:
model.save('./CNN_catordog.h5')